## Thursday 2/11/2017 : 

I started by cloning/downloading the git repository of openCitations locally and start analyzing the general structure/organization of the project. My main attention today was on the ‘website/‘ directory which defines the web pages of OpenCitations. Starting from the server module ‘oc.py’, I studied and commented the code and its main parts. This analyzes was essential to better understand the server code and web.py functionalities since I have never used it before.


Once I have understood the basic/essential parts, I re-adapted the configuration file according to my local URLs. All the necessary modules to run the python server were successfully installed under the python2.7 version. The website runs correctly locally and I can use it to formulate some tests on it.

---

## Friday 3/11/2017 :

Analyzing the .html files and generally all the static modules of the website. I have created a subdirectory inside my repository [doc/](doc/) with a *oc-rep-doc.md* file: I will use this file to write down notes and a road map on approaching some problems. Since my focus right now is on the website part, I started by adding some notes along with the adjustments that we might or eventually should do. Part of these notes are based on tests I am doing with the modification of the actual code.


One of our objects is creating a query interface which will operate on our corpus. Therfore as a possible first modification to the interface I have added a search box in the index menu, and the possible input right now for experimental reasons can be any type of text. The idea is to redirect the search operation as a GET request with a query, maybe to the sparql page.

---

## Monday 6/11/2017 :

I started reading some SPARQL documentations specially regarding the query formats to adapt for our case, giving our general textual input. I have tested different SPARQL query approaches on some external datasets. I think the idea is to apply the searching input as a filter, e.g:  `FILTER regex(?title, "SomeText”)`, in case the searching input is directed to the titles.

I have created a separated session (page) on the web interface. In this case, after typing the textual content inside the searching box, the user will be redirected to the dedicated page. The idea is to visualize the eventual results retrieved in a structured form. This page will be also designed in the future for considering advanced and additional filtering/searching parameters.

---

## Tuesday 7/11/2017:

I am trying to better understand blazegraph (https://www.blazegraph.com/) especially the the bigdata full text indexing and searching features provided. Since this might be a possible approach for general textual searching. We need to take in consideration the final performances we achieve with such approach, despite for example directing the actual queries to specific fields (e.g: DOI, Author …), after applying some filters and regular expression mapping to the actual searching input. 

I have created a python module to test the SPARQLWrapper library and functionalities. A similar python script should be called at the server side in order to perform the SPARQL query on our dataset and generate the results in JSON format to the client side. 

---

## Wednesday 8/11/2017:

I need to start doing some queries locally on my machine with a OCC data dump. My aim is to retrieve one of the initial data dumps made and uploaded on figshare (https://figshare.com/), like the September 24, 2016 dump, and test some sparql queries through blazegraph on it. I am having some technical issues regard the unpacking of all the necessary files from the .dar archives. Hopefully with the help of Silvio Peroni i will fix this problem and have all the data correctly stored locally on my machine. Meanwhile I was still testing some sparql possible queries we could possibly apply. 


I continued to deepen the blazegraph documentation, and testing its interface components/features. I am using some tutorials and samples for a better comprehension.


---

## Thursday 9/11/2017:

I managed to handle the technical problems while generating all the files needed from the .dar archives, with the help of Silvio Peroni. So right now i have the *blazegraph/sparql* endpoint running on my 3000 localhost port. I can use this sparql endpoint as a target for my queries. Through python I am testing two approaches: 1) with the Pymantic library (suggested as an example from blazegraph site). 2) SparqlWrapper.

I think SparqlWrapper is a much better solution, since its also highly used in a variety of sparql applications. 
Beside testing, I am deepen my study in the OCO list of ontologies. Trying to figure out the relations I mostly need when applying the queries.

---

## Friday 10/11/2017:

I have made a python module that should handle my regular expressions calls. For instance one important rule should detect a DOI from the textual input in the search box, in this case I studied the DOI structure/format and the possible regular expression rules which can match any given DOI. I will call this module and its inner defined methods, for any other possible textual matching problem encountered, besides the searching operations.

In [5]:
#A possible regular expression form for DOI detection:  
import re
match_obj = re.match( r'10.\d{4,9}/[-._;()/:A-Z0-9]+$', "my_text", re.M|re.I)

The textual matching phase will be a pre-step before calling the sparql endpoint and applying a query to the id field. I have tested some queries regarding how to correctly getting the actual DOI values from the bibliographic resources, and comparing those with a value given. 

---

## Monday 13/11/2017:

I have been testing some of the full text search operations through the blazegraph sparql endpoint. Basically, I have formulated queries which will search for all the literals that have a value equal to the searching field. For instance to find a particular DOI:

In [ ]:
select ?subj ?p ?lit 
where {
        ?lit bds:search "10.1002/cncr.28414" .
        ?lit bds:matchAllTerms "true" .
        ?subj ?p ?lit .  
}

We have arranged a meeting with all the DASPLAB members and discussed the possible issues and future applications. I have continued the rest of the day formulating some tests on blazegraph sparql endpoint. 

---

## Tuesday 14/11/2017:

I have created new additional modules to include inside the website subdirectory:
* sparql_conf.json : this file will contain some configuration/variables needed to the sparql_endpoint when executing the queries. (ex: the prefixes). Right now it contains the OC parameters, for instance the prefixes OCO (ontologies prefixes) this parameters could be reconfigured in case we need to manage other systems and datasets.
* ~~src/sparql_endpoint.py : this module creats the sparql endpoint object initialized with the parameters of sparql_conf.json, and it will be called for the application of queries, specially when called from the searching operations. Right now the main method in it, is ‘do_fulltext_query’ which will apply a full text search and return the results in json format.~~ 
> This approach has been changed. See **Wednesday 15/11/2017**.
* html-template/search.html, src/search.js : these 2 modules are used to build the search section/page. 

---

## Wednesday 15/11/2017:

I have started developing the actual Search/ page interface and layout (working with search.html and search.js). The layout of the page is similar to the following scheme. When applying a search operation the sparql query returns results in a json format. The json data will populate a table object, which will be visualized in the searching results container. The filters on the left side will enable a further re-elaboration on the results. The header will contain operations/visualizations to apply on the results, e.g: a sort operation.

![Image](img/search-layout.png)

**A necessary re-adaption:** I must move the sparql queries elaborations from the .py module to a .js one. This operation is necessary, since we need to build a general processing approach which might be used by any other system (Besides OC). This behavior is possible with the addition of a re-configurable json file which defines all the configurations and possible queries.

---

## Thursday 16/11/2017 :

In order to let the searching procedure be re-adaptable for any system, I have built a .json file containing the parameters and configurations variables which could be defined by the user: 
* sparql_endpoint
* a list of the prefixes (to add before the actual querie)
* a list of rules: each rule is defined by three parameters: the id, the regex rule and the corresponding textual query to apply. Inside the query we insert a VAR tag which will be replaced with the searching text.

I have defined a .js method to call sparql and apply the query, along with .js method that handles the regular expressions and assign the correct query, as its defined in the rules field inside the configuration file.
I am having some issues when calling sparql through .js, since it seems i can't get .json result when calling a sparql endpoint which resides on a different domain (the server should be allowed to do that: *Access-Control-Allow-Origin*). Therefor the possible solution is to retrieve the GET result in a .jsonp format, which still is not giving me the expected results.

---

## Friday 17/11/2017:

I have managed to solve yesterday problem (the Access-Control-Allow-Origin issue) with the help of Silvio: The OC server already allows the calls on external origins, therefore the solution was to redirect all the GET calls to the server which will further apply a query to the dataset.
Now the procedure to accomplish through javascript is almost all defined. Any search call will go through this steps/methods :   
* 1\. **_get_search_conf():** to get all the search configurations, the sparql endpoind url, the prefixes and the rules (see Thursday)
* 2\. Build the query string in turtle format:
	* 2.1\. **turtle_prefixes():** builds a string combining all the prefixes of the config file, in a turtle format (*PREFIX [prefix.name] : <[prefix.url]>* ) 
	* 2.2\. **turtle_query(text):** to accomplish this operation we go through this steps:
		* 2.2.1\.  iterate the list of rules and check which regex rule matches the *text* parameter 
		* 2.2.2\.  return the corresponding string of the query in turtle format and replace the *VAR* tag inside it with *text*. (e.g: if the query looks for a DOI, the variable *text* to look for could be equal to '10.1002/cncr.28414' )    
* 3\. **do_search_query(query_str):** apply the query and get the results in json format.
* 4\. **build_tab_from_json(json_data):** builds a structured table for the results obtained from the previous step.